In [1]:
import os
import json
from pathlib import Path
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from model.dgn.gmdn import GMDN
from model.util.graph_expert_emission import GraphExpertEmission

In [2]:
folder = f'RESULTS_SIR/RESULTS_SIR_GMDN/MGDN_erdos_renyi_100_0.01@0.05@0.1@0.2_assessment/HOLDOUT_ASS/'
ckpt = Path(folder/ Path('best_checkpoint.pth'))
config_file = Path(folder/ Path('HOLDOUT_MS') / Path('winner_config.json'))

with open(config_file, 'r') as f:
    config_data = json.load(f)
config_data['config']['supervised_config']

# Backward compatibility with old code
config_data['config']['supervised_config']['neighborhood_aggregation'] = 'add'

print(config_data)

FileNotFoundError: [Errno 2] No such file or directory: 'RESULTS_SIR/RESULTS_SIR_GMDN/MGDN_erdos_renyi_100_0.01@0.05@0.1@0.2_assessment/HOLDOUT_ASS/HOLDOUT_MS/winner_config.json'

In [8]:
from data.dataset import ErdosRenyiDataset

dataset = ErdosRenyiDataset('DATA/ERDOS_RENYI', 'erdos_renyi', size=100, connectivity="0.01@0.05@0.1@0.2")

In [11]:
model = GMDN(dataset.dim_node_features, dataset.dim_edge_features, dataset.dim_target, GraphExpertEmission, config_data['config']['supervised_config'])

In [12]:
device = 'cpu'
model.load_state_dict(torch.load(ckpt)['model_state'])
model.eval()
model.to(device)

GMDN(
  (layer): GMDNEncoder(
    (emission): GraphExpertEmission(
      (output_activation): Identity()
      (node_transform): Identity()
      (final_transform): Linear(in_features=128, out_features=20, bias=False)
    )
    (transition): GMDNTransition(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=5, out_features=64, bias=False)
          (1): ReLU()
          (2): Linear(in_features=64, out_features=64, bias=False)
          (3): ReLU()
        )
        (1): GMDNConv(
          (nn): Sequential(
            (0): Linear(in_features=64, out_features=64, bias=False)
            (1): ReLU()
            (2): Linear(in_features=64, out_features=64, bias=False)
            (3): ReLU()
          )
        )
      )
      (fc_global): Linear(in_features=128, out_features=64, bias=False)
      (out): Linear(in_features=64, out_features=10, bias=True)
    )
  )
)

In [13]:
from torch_geometric.data import Batch

# Same beta and gamma on all nodes
data_list = [(d.x[0,-2]/d.x[0,-1], d) for d in dataset]
data_list.sort(key=lambda x: x[0]) # sort by R0
dataset = Batch.from_data_list([d[1] for d in data_list])
dataset.to(device)

KeyboardInterrupt: 

In [ ]:
from torch.distributions import *

def binomial_mixture_pdf(labels, coef, n, p):
    
    # Assume 1 for now
    no_features = 1 
    if len(n.shape) == 2:
        n = n.unsqueeze(2)  # add output feature dim
        # distr_params is now [samples, no_experts, 1=no_features]
    if len(p.shape) == 2:
        p = p.unsqueeze(2)  # add output feature dim     
    mix = Categorical(coef)
    comp = Independent(Binomial(n, p), 1)
    pmm = MixtureSameFamily(mix, comp) 
    
    if len(labels.shape) == 1:
        labels = labels.unsqueeze(1)  # add output feature dim

    return pmm.log_prob(labels.float()).exp()

In [ ]:
from torch_geometric.nn.glob import global_mean_pool
y = dataset.y

no_points = 100

beta_space = np.linspace(0, 1, no_points)
gamma_space = np.linspace(0.1, 1, no_points)

y_space = torch.tensor(np.linspace(y_min, y_max, no_points))

r0_subspace_graph = []
prob_subspace_graph = []
coef_subspace_graph = []
p_subspace_graph = []

for graph_id in range(100):  # no of graphs

    r0_space = None
    prob_space = None
    coef_space = None
    p_space = None

    # Take the graph
    g = data_list[graph_id][1]
    
    for i in range(no_points):
        for j in range(no_points):
            r0 = torch.tensor([beta_space[i]/gamma_space[j]]).unsqueeze(0)
            g.x[:, 1:-1] = torch.tensor([r0, beta_space[i], gamma_space[j]]).float().unsqueeze(0)
            g.x[:, -1] = data_list[0][1].x[:,-1]

            x = Batch.from_data_list([g])
            *rest, (params_g, coef_g) = model(x)
            n_g, p_g = params_g
            n_g, p_g, coef_g = n_g.detach(), p_g.detach(), coef_g.detach()
            prob = binomial_mixture_pdf(y_space, coef_g, n_g, p_g).unsqueeze(1)

            if r0_space is None:
                r0_space = r0
            else:
                r0_space = torch.cat((r0_space, r0), dim=0)

            if prob_space is None:
                prob_space = prob
            else:
                prob_space = torch.cat((prob_space, prob), dim=1)

            if coef_space is None:
                coef_space = coef_g
            else:
                coef_space = torch.cat((coef_space, coef_g), dim=0)

            if p_space is None:
                p_space = p_g
            else:
                p_space = torch.cat((p_space, p_g), dim=0)

    # Use no_points instead of no_points*no_points
    prob_subspace_graph.append(prob_space[:, ::no_points])
    coef_subspace_graph.append(coef_space[::no_points, :])
    p_subspace_graph.append(p_space[::no_points, :])
    r0_subspace_graph.append(r0_space[::no_points, :])

In [14]:
# Store them to avoid recomputing them every time
torch.save(dict(prob_subspace_graph=prob_subspace_graph, coef_subspace_graph=coef_subspace_graph, p_subspace_graph=p_subspace_graph, r0_subspace_graph=r0_subspace_graph), 'ER_100_visualization_results.torch')

In [17]:
import pandas as pd

no_points = 100
d = torch.load('ER_100_visualization_results.torch')

prob_subspace_graph = d['prob_subspace_graph']
coef_subspace_graph = d['coef_subspace_graph']
p_subspace_graph = d['p_subspace_graph']
r0_subspace_graph = d['r0_subspace_graph']

p_df = pd.DataFrame(columns=['graph_id', r'$\beta/\gamma$', 'p', 'expert'])
coef_df = pd.DataFrame(columns=['graph_id', r'$\beta/\gamma$', 'weight', 'expert'])
prob_df = pd.DataFrame(columns=['graph_id', r'$\beta/\gamma$', r'$P(y|g,\beta,\gamma)$', 'expert'])

for i in range(100):  # no of graphs
    if (i%10) == 0 or i == 99:
        print(i)
    r0_list = r0_subspace_graph[i].squeeze().numpy()
    for j in range(no_points):
        r0 = r0_list[j]
        for expert in range(5):
            new_row = {'graph_id': str(i), r'$\beta/\gamma$': r0, 'p': float(p_subspace_graph[i][j, expert]), 'expert': str(expert+1)}
            p_df = p_df.append(new_row, ignore_index=True)
            
            new_row = {'graph_id': str(i), r'$\beta/\gamma$': r0, 'weight': float(coef_subspace_graph[i][j, expert]), 'expert': str(expert+1)}
            coef_df = coef_df.append(new_row, ignore_index=True)
            
            new_row = {'graph_id': str(i), r'$\beta/\gamma$': r0, r'$P(y|g,\beta,\gamma)$': float(prob_subspace_graph[i][j, expert]), 'expert': str(expert+1)}
            prob_df = prob_df.append(new_row, ignore_index=True)     

0
10
20
30
40
50
60
70
80
90
99


In [18]:
prob_df.to_pickle('OUTPUTS/ER_100_prob_df.pkl')
coef_df.to_pickle('OUTPUTS/ER_100_coef_df.pkl')
p_df.to_pickle('OUTPUTS/ER_100_p_df.pkl')

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
prob_df = pd.read_pickle('OUTPUTS/ER_100_prob_df.pkl')
coef_df = pd.read_pickle('OUTPUTS/ER_100_coef_df.pkl')
p_df = pd.read_pickle('OUTPUTS/ER_100_p_df.pkl')

In [ ]:
p_df = p_df.rename(columns={"expert": "sub-net"})

plt.rcParams.update({'font.size': 14})
sns.lineplot(x=r'$\beta/\gamma$', y='p', hue='sub-net', data=p_df, units='graph_id', estimator=None, lw=0.5, palette='colorblind')
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('OUTPUTS/ER_100_binomial_distribution.eps')

In [ ]:
coef_df = coef_df.rename(columns={"expert": "sub-net"})

plt.rcParams.update({'font.size': 14})
sns.lineplot(x=r'$\beta/\gamma$', y='weight', hue='sub-net', data=coef_df, units='graph_id', estimator=None, lw=0.5, palette='colorblind')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('OUTPUTS/ER_100_weight_distribution.eps')